In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn import metrics
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from six.moves import cPickle as pickle

Using TensorFlow backend.


In [2]:
data=pd.read_csv('train.csv')
test=pd.read_csv("test.csv")
datadiff=pd.read_csv("extradata.csv")
testdiff=pd.read_csv("extratest.csv")
extra = pd.read_csv("extra_infos.csv")

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data=pd.concat([data,datadiff.iloc[:,1:3]],axis=1)
test=pd.concat([test,testdiff.iloc[:,1:3]],axis=1)

In [4]:
import random
kt=data[data.listen_type==1]
kt1=data[data.listen_type==0]
kt.reset_index(drop=True, inplace=True)
kt1.reset_index(drop=True, inplace=True)
kt=kt.sort_values(['release_date'], ascending=[False])
kt.reset_index(drop=True, inplace=True)
rows = np.random.choice(kt[0:800000].index.values, 100000)
sampled_df = kt.ix[rows]
kt = kt.drop(kt.index[rows])
kt.reset_index(drop=True, inplace=True)
rows = np.random.choice(kt[0:800000].index.values, 100000)
sampled_df1 = kt.ix[rows]
kt = kt.drop(rows)
kt.reset_index(drop=True, inplace=True)
data.reset_index(drop=True, inplace=True)
sampled_df.reset_index(drop=True, inplace=True)
sampled_df1.reset_index(drop=True, inplace=True)
data=pd.concat([kt,kt1,sampled_df,sampled_df1],axis=0)
data.reset_index(drop=True, inplace=True)
data= data.sample(frac=1).reset_index(drop=True)
data.shape
del kt
del kt1
del sampled_df1
del sampled_df

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  del sys.path[0]


In [5]:
df43 = pd.DataFrame(data, columns = ['genre_id', 'is_listened'])
df45 = pd.DataFrame(data, columns = ['media_id', 'is_listened'])
df46=  pd.DataFrame(data, columns = ['album_id', 'is_listened'])
df47=  pd.DataFrame(data, columns = ['artist_id', 'is_listened'])
df51=  pd.DataFrame(data, columns = ['user_id', 'is_listened'])
df53=  pd.DataFrame(data, columns = ['release_date', 'is_listened'])
df55=  pd.DataFrame(data, columns = ['time1', 'is_listened'])

In [6]:
#TO create fractional features. First groupby ids and then calculate mean of is listened for every level of id
#and then map it on data using dictionary creation
count1=df43.groupby(['genre_id'],as_index=False).agg({'is_listened':'mean'})
df44=pd.Series(count1.is_listened.values,index=count1.genre_id).to_dict()
data["B"] = data["genre_id"].map(df44)
test["B"] = test["genre_id"].map(df44)


count2=df45.groupby(['media_id'],as_index=False).agg({'is_listened':'mean'})
df48=pd.Series(count2.is_listened.values,index=count2.media_id).to_dict()
data["C"] = data["media_id"].map(df48)
test["C"] = test["media_id"].map(df48)

count3=df46.groupby(['album_id'],as_index=False).agg({'is_listened':'mean'})
df49=pd.Series(count3.is_listened.values,index=count3.album_id).to_dict()
data["D"] = data["album_id"].map(df49)
test["D"] = test["album_id"].map(df49)
data.head()
count4=df47.groupby(['artist_id'],as_index=False).agg({'is_listened':'mean'})
df50=pd.Series(count4.is_listened.values,index=count4.artist_id).to_dict()
data["E"] = data["artist_id"].map(df50)
test["E"] = test["artist_id"].map(df50)

count5=df51.groupby(['user_id'],as_index=False).agg({'is_listened':'mean'})
df52=pd.Series(count5.is_listened.values,index=count5.user_id).to_dict()
data["F"] = data["user_id"].map(df52)
test["F"] = test["user_id"].map(df52)

count6=df53.groupby(['release_date'],as_index=False).agg({'is_listened':'mean'})
df54=pd.Series(count6.is_listened.values,index=count6.release_date).to_dict()
data["G"] = data["release_date"].map(df54)
test["G"] = test["release_date"].map(df54)

count6=df53.groupby(['release_date'],as_index=False).agg({'is_listened':'mean'})
df54=pd.Series(count6.is_listened.values,index=count6.release_date).to_dict()
data["G"] = data["release_date"].map(df54)
test["G"] = test["release_date"].map(df54)

count7=df55.groupby(['time1'],as_index=False).agg({'is_listened':'mean'})
df56=pd.Series(count7.is_listened.values,index=count7.time1).to_dict()
data["H"] = data["time1"].map(df56)
test["H"] = test["time1"].map(df56)

In [7]:
del count1
del count2
del count3
del count4
del count5
del count6
del df43
del df44
del df45
del df46
del df47
del df48
del df49
del df50
del df51
del df52

In [8]:
#pivot table for clustering of similar users. Till now only used genre id for cluster
#print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()
matrix = data.pivot_table(index=['user_id'], columns=['genre_id'], values='is_listened')
from sklearn.cluster import KMeans
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usergencluster"] = data["user_id"].map(mat2)
test["usergencluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

data['album_id_1']=data['album_id']
test['album_id_1']=test['album_id']
mmo3=pd.DataFrame(data['album_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.album_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[1500:,0])
data['album_id_1'].loc[data['album_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['album_id_1'], values='is_listened')
cluster = KMeans(n_clusters=40)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["useralbcluster"] = data["user_id"].map(mat2)
test["useralbcluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

data['artist_id_1']=data['artist_id']
test['artist_id_1']=test['artist_id']
mmo3=pd.DataFrame(data['artist_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.artist_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[1700:,0])
data['artist_id_1'].loc[data['artist_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['artist_id_1'], values='is_listened')
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userartcluster"] = data["user_id"].map(mat2)
test["userartcluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

matrix=0
data['release_date_1']=data['release_date']
test['release_date_1']=test['release_date']
mmo3=pd.DataFrame(data['release_date_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.release_date_1.values)],axis=1)
mmo31=list(mmo3.iloc[1000:,0])
data['release_date_1'].loc[data['release_date_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['release_date_1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userdatecluster"] = data["user_id"].map(mat2)
test["userdatecluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

matrix=0
data['time1_1']=data['time1']
test['time1_1']=test['time1']
mmo3=pd.DataFrame(data['time1_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.time1_1.values)],axis=1)
mmo31=list(mmo3.iloc[:,0])
#data['time1_1'].loc[data['time1_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['time1_1'], values='is_listened')
cluster = KMeans(n_clusters=4)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usertimecluster"] = data["user_id"].map(mat2)
test["usertimecluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()
del matrix
del matrix1
del mat2

0:06:23.112421


/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0:02:33.610588
0:03:03.267682
0:01:14.600236
0:00:07.617969


In [10]:
data['med'] = data['media_id']
test['med'] = test['media_id']
data.shape

(7570895, 34)

In [12]:
data.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,usergencluster,album_id_1,useralbcluster,artist_id_1,userartcluster,release_date_1,userdatecluster,time1_1,usertimecluster,med
0,2,1479661767,16392025,1531010,0,20120130,0,0,181,0,...,43,-1,21,58801,43,20120130,8,17,3,16392025
1,0,1478557511,121275010,12670134,4,20160325,0,0,176,1,...,42,-1,19,1590752,47,20160325,2,22,0,121275010
2,0,1478396473,132911084,14137030,3,20160927,0,0,141,0,...,18,-1,18,1191615,48,20160927,2,1,0,132911084
3,25201,1478634042,12266946,1127221,0,20090223,1,0,295,0,...,27,-1,24,-1,10,-1,1,19,2,12266946
4,0,1478550725,122273422,12790880,0,20160429,0,0,216,0,...,43,12790880,18,6920971,5,20160429,2,20,2,122273422


In [13]:
data['media_id_1']=data['media_id']
test['media_id_1']=test['media_id']
mmo3=pd.DataFrame(data['media_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.media_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[3000:,0])
data['media_id_1'].loc[data['media_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['media_id_1'], values='is_listened')
cluster = KMeans(n_clusters=50)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usermedcluster"] = data["user_id"].map(mat2)
test["usermedcluster"]=test["user_id"].map(mat2)
print(datetime.datetime.now()-start_time)
start_time = datetime.datetime.now()

0:11:35.386803


In [10]:
#from six.moves import cPickle as pickle
#file_name = "music.pickle"
#data.to_pickle(file_name)
test.to_pickle("test.pickle")

In [2]:
data = pd.read_pickle("music.pickle")
test = pd.read_pickle("test.pickle")

In [4]:
from scipy.sparse import csr_matrix
extra = pd.read_csv("extra_infos.csv")
extra = extra.ix[: , 2:4] #Choosing only the media_id and sng_name variables
from sklearn.feature_extraction.text import CountVectorizer
tokens = '\\S+(?=\\s+)'   #Regex
extra.sng_title.fillna('' , inplace = True)
vec = CountVectorizer(token_pattern = tokens , max_features = 30)
tf = vec.fit_transform(extra["sng_title"])
j = csr_matrix.todense(tf)
j = pd.DataFrame(j , index = extra["media_id"])  #this gives the final features 
j.reset_index(inplace=True)
j.head()

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


,media_id,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,213952,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,223014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,237198,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,238380,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,244881,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
j.shape

(453310, 21)

In [48]:
data.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,media_id_135949700,media_id_135949706,media_id_135949708,media_id_135949710,media_id_136017244,media_id_136662032,usertimecluster_0,usertimecluster_1,usertimecluster_2,usertimecluster_3
0,0,1478950200,-1,-1,0,-1,0,0,225,0,...,0,0,0,0,0,0,1,0,0,0
1,-1,1479865789,-1,-1,0,-1,0,0,255,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1478156053,125890431,13246617,0,20160704,0,0,179,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1480164382,-1,-1,2,-1,0,0,161,0,...,0,0,0,0,0,0,1,0,0,0
4,-1,1479291089,-1,-1,1,-1,2,1,180,1,...,0,0,0,0,0,0,0,0,1,0


In [33]:
data = pd.merge(data, j, how='left', on=['media_id'])
test = pd.merge(test, j, how='left', on=['media_id'])

In [28]:
#data1.columns.tolist()
data.shape

(7570688, 55)

In [34]:
mmo3=pd.DataFrame(data['media_id'].value_counts())
mmo98=pd.DataFrame(test['media_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo3.index.values))
test['media_id'].loc[test['media_id'].isin(list(unmatched)[65:])]=-2
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.media_id.values)],axis=1)
df=pd.Series(mmo3.iloc[:,1].values,index=mmo3.iloc[:,0].values).to_dict()
data["media_id_freq"] = data["media_id"].map(df)
test["media_id_freq"] = test["media_id"].map(df)
mmo31=list(mmo3.iloc[100:,0])
data['media_id'].loc[data['media_id'].isin(mmo31)]=-1
test['media_id'].loc[test['media_id'].isin(mmo31)]=-1


mmo3=pd.DataFrame(data['album_id'].value_counts())
mmo98=pd.DataFrame(test['album_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo3.index.values))
test['album_id'].loc[test['album_id'].isin(list(unmatched)[65:])]=-2
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.album_id.values)],axis=1)
df=pd.Series(mmo3.iloc[:,1].values,index=mmo3.iloc[:,0].values).to_dict()
data["album_id_freq"] = data["album_id"].map(df)
test["album_id_freq"] = test["album_id"].map(df)
mmo31=list(mmo3.iloc[100:,0])
data['album_id'].loc[data['album_id'].isin(mmo31)]=-1
test['album_id'].loc[test['album_id'].isin(mmo31)]=-1

mmo4=pd.DataFrame(data['artist_id'].value_counts())
mmo98=pd.DataFrame(test['artist_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo4.index.values))
test['artist_id'].loc[test['artist_id'].isin(list(unmatched)[40:])]=-2
mmo4=pd.concat([pd.DataFrame(mmo4.index.values),pd.DataFrame(mmo4.artist_id.values)],axis=1)
df=pd.Series(mmo4.iloc[:,1].values,index=mmo4.iloc[:,0].values).to_dict()
data["artist_id_freq"] = data["artist_id"].map(df)
test["artist_id_freq"] = test["artist_id"].map(df)
mmo41=list(mmo4.iloc[100:,0])
data['artist_id'].loc[data['artist_id'].isin(mmo41)]=-1
test['artist_id'].loc[test['artist_id'].isin(mmo41)]=-1

      
mmo5=pd.DataFrame(data['genre_id'].value_counts())
mmo98=pd.DataFrame(test['genre_id'].value_counts())
#unmatched = set(list(mmo98.index.values))-set(list(mmo5.index.values))
#test['genre_id'].loc[test['genre_id'].isin(unmatched)]=-2
mmo5=pd.concat([pd.DataFrame(mmo5.index.values),pd.DataFrame(mmo5.genre_id.values)],axis=1)
df=pd.Series(mmo5.iloc[:,1].values,index=mmo5.iloc[:,0].values).to_dict()
data["genre_id_freq"] = data["genre_id"].map(df)
test["genre_id_freq"] = test["genre_id"].map(df)
mmo51=list(mmo5.iloc[50:,0])
data['genre_id'].loc[data['genre_id'].isin(mmo51)]=-1
test['genre_id'].loc[test['genre_id'].isin(mmo51)]=-1

mmo6=pd.DataFrame(data['context_type'].value_counts())
mmo6=pd.concat([pd.DataFrame(mmo6.index.values),pd.DataFrame(mmo6.context_type.values)],axis=1)
mmo61=list(mmo6.iloc[25:,0])
data['context_type'].loc[data['context_type'].isin(mmo61)]=-1
test['context_type'].loc[test['context_type'].isin(mmo61)]=-1


mmo7=pd.DataFrame(data['release_date'].value_counts())
mmo98=pd.DataFrame(test['release_date'].value_counts())
#unmatched = set(list(mmo98.index.values))-set(list(mmo7.index.values))
#test['release_date'].loc[test['release_date'].isin(unmatched)]=-1
mmo7=pd.concat([pd.DataFrame(mmo7.index.values),pd.DataFrame(mmo7.release_date.values)],axis=1)
mmo71=list(mmo7.iloc[50:,0])
data['release_date'].loc[data['release_date'].isin(mmo71)]=-1
test['release_date'].loc[test['release_date'].isin(mmo71)]=-1

In [35]:
del mmo3
del mmo31
del mmo4
del mmo5
del mmo6
del mmo7
del mmo41
del mmo51
del mmo61
del mmo71
del mmo98

In [36]:
target=data['is_listened']
del data['is_listened']
test2=test.iloc[:,1:] 
data.head()
test2.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,14,15,16,17,18,19,media_id_freq,album_id_freq,artist_id_freq,genre_id_freq
0,50,1478104371,-1,-1,1,-1,0,0,542,1,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,20.0,1111.0,79179
1,2744,1479317140,-1,-1,1,-1,0,0,307,1,...,NaN,NaN,NaN,NaN,NaN,NaN,441.0,1363.0,5537.0,84870
2,2744,1479546361,-1,-1,1,-1,0,0,307,1,...,NaN,NaN,NaN,NaN,NaN,NaN,441.0,1363.0,5537.0,84870
3,2744,1478457729,-1,-1,1,-1,2,1,265,1,...,NaN,NaN,NaN,NaN,NaN,NaN,176.0,1363.0,5537.0,84870
4,2744,1480448560,-1,-1,1,-1,2,1,356,1,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,1363.0,5537.0,84870


In [37]:
data.shape

(7570688, 58)

In [38]:
data=pd.concat([data,test2],axis=0)
data=data.fillna(0)

In [39]:
data.shape

(7590606, 58)

In [40]:
dummie1 = pd.get_dummies(data['context_type'], prefix='context_type', prefix_sep='_')
dummie2 = pd.get_dummies(data['platform_name'], prefix='platform_name', prefix_sep='_')
dummie3 = pd.get_dummies(data['platform_family'], prefix='platform_family', prefix_sep='_')
dummie4 = pd.get_dummies(data['listen_type'], prefix='listen_type', prefix_sep='_')
dummie5 = pd.get_dummies(data['user_gender'], prefix='user_gender', prefix_sep='_')
dummie6 = pd.get_dummies(data['genre_id'], prefix='genre_id', prefix_sep='_')
dummie7 = pd.get_dummies(data['album_id'], prefix='album_id', prefix_sep='_')
dummie8 = pd.get_dummies(data['release_date'], prefix='album_id', prefix_sep='_')

dummie16 = pd.get_dummies(data['media_id'], prefix='media_id', prefix_sep='_')
#dummie9 = pd.get_dummies(fulldata['user_id'], prefix='user_id', prefix_sep='_',sparse=True)
dummie9 = pd.get_dummies(data['usergencluster'], prefix='usergencluster', prefix_sep='_')
dummie10 = pd.get_dummies(data['artist_id'], prefix='artist_id', prefix_sep='_')
dummie11 = pd.get_dummies(data['usermedcluster'], prefix='usermedcluster', prefix_sep='_')
dummie12= pd.get_dummies(data['useralbcluster'], prefix='useralbcluster', prefix_sep='_')
dummie13= pd.get_dummies(data['userartcluster'], prefix='userartcluster', prefix_sep='_')
dummie14= pd.get_dummies(data['userdatecluster'], prefix='userdatecluster', prefix_sep='_')
dummie15= pd.get_dummies(data['usertimecluster'], prefix='usertimecluster', prefix_sep='_')

In [41]:
data=pd.concat([data,dummie10,dummie1,dummie2,dummie3,dummie4,dummie5,dummie6,dummie7,dummie8,dummie9,dummie11,dummie12,dummie13,dummie14,dummie16,dummie15],axis=1)
data.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,media_id_135949700,media_id_135949706,media_id_135949708,media_id_135949710,media_id_136017244,media_id_136662032,usertimecluster_0,usertimecluster_1,usertimecluster_2,usertimecluster_3
0,0,1478950200,-1,-1,0,-1,0,0,225,0,...,0,0,0,0,0,0,1,0,0,0
1,-1,1479865789,-1,-1,0,-1,0,0,255,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1478156053,125890431,13246617,0,20160704,0,0,179,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1480164382,-1,-1,2,-1,0,0,161,0,...,0,0,0,0,0,0,1,0,0,0
4,-1,1479291089,-1,-1,1,-1,2,1,180,1,...,0,0,0,0,0,0,0,0,1,0


In [43]:
del dummie1
del dummie2
del dummie3
del dummie4
del dummie5
del dummie6
del dummie7
del dummie8
del dummie9
del dummie10
del dummie11
del dummie12
del dummie13
del dummie14

In [47]:
test.head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,...,14,15,16,17,18,19,media_id_freq,album_id_freq,artist_id_freq,genre_id_freq
0,0,50,1478104371,-1,-1,1,-1,0,0,542,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,20.0,1111.0,79179
1,1,2744,1479317140,-1,-1,1,-1,0,0,307,...,NaN,NaN,NaN,NaN,NaN,NaN,441.0,1363.0,5537.0,84870
2,2,2744,1479546361,-1,-1,1,-1,0,0,307,...,NaN,NaN,NaN,NaN,NaN,NaN,441.0,1363.0,5537.0,84870
3,3,2744,1478457729,-1,-1,1,-1,2,1,265,...,NaN,NaN,NaN,NaN,NaN,NaN,176.0,1363.0,5537.0,84870
4,4,2744,1480448560,-1,-1,1,-1,2,1,356,...,NaN,NaN,NaN,NaN,NaN,NaN,15.0,1363.0,5537.0,84870


In [46]:
predictors = data.iloc[:6000000,38:].as_matrix()# assign feature dataframe to predictors removing the target column
target1 = to_categorical(target)# assign target dataframe to y
n_cols = predictors.shape[1]
predictors.shape

MemoryError: 

In [18]:
train_size = len(data) - len(test) - 200000
train_size
valid_size = len(data) - len(test)
valid_size

7570894

In [41]:
n_cols = predictors.shape[1]

In [42]:
early_stopping_monitor = EarlyStopping(patience = 2)

# Model
model = Sequential()
model.add(Dense(96 , activation = 'relu' , input_shape = (n_cols,)))
#model.add(Dropout(0.4))
model.add(Dense(96 , activation = 'relu'))
#model.add(Dropout(0.4))
model.add(Dense(56 , activation = 'relu'))
#model.add(Dropout(0.4))
model.add(Dense(56 , activation = 'relu'))
#model.add(Dropout(0.4))
model.add(Dense(56 , activation = 'relu'))
model.add(Dense(42 , activation = 'relu'))
model.add(Dense(14 , activation = 'relu'))
#model.add(Dropout(0.4))

model.add(Dense(2 , activation = 'softmax'))

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [45]:
model.fit(predictors , target1[0:6000000] , validation_data = [validset,target1[train_size+1:]] , epochs = 1 , callbacks = [early_stopping_monitor])

Train on 6000000 samples, validate on 199999 samples
Epoch 1/1
6000000/6000000 [==============================] - 654s - loss: 0.5141 - acc: 0.7461 - val_loss: 0.5055 - val_acc: 0.7513


In [62]:
print(n_cols)
p_tr = model.predict_proba(predictors, verbose=0)[:,1]
p_tr1 = model.predict_proba(predictors1, verbose=0)[:,1]

818


In [67]:
#k_sub1 = pd.DataFrame(p_tr, columns=['keras_nd_1e_6l'])
#k_sub2 = pd.DataFrame(p_tr1, columns=['keras_nd_1e_6l'])
sub=pd.concat([k_sub1,k_sub2],axis=0)

In [68]:
len(sub)

7570688

In [69]:
sub.to_csv("keras_nd_1e_train.csv")

In [70]:
model_json = model.to_json()
with open("keras_nd_1e_2505.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("keras_nd_1e_2505.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
valid_preds = model.predict_proba(predictors[train_size+1:valid_size], verbose=0)
valid_preds = valid_preds[:, 1]
roc = metrics.roc_auc_score(target1[train_size+1:valid_size,1], valid_preds)
roc

0.78659001653668625

In [22]:
preds = model.predict_proba(predictors[valid_size:], verbose=0)[:, 1]
submission = pd.DataFrame(preds, columns=['is_listened'])
submission.to_csv('Keras_21054_b2_100_1.csv')

In [24]:
preds

array([ 0.62940764,  0.63872761,  0.42107567, ...,  0.8877719 ,
        0.77361989,  0.86544377], dtype=float32)

In [27]:
early_stopping_monitor = EarlyStopping(patience = 2)

# Model
model1 = Sequential()
model1.add(Dense(96 , activation = 'relu' , input_shape = (n_cols,)))
model1.add(Dropout(0.4))
model1.add(Dense(96 , activation = 'relu'))
model1.add(Dropout(0.4))
model1.add(Dense(56 , activation = 'relu'))
model1.add(Dropout(0.4))
model1.add(Dense(56 , activation = 'relu'))
model1.add(Dropout(0.4))
model1.add(Dense(56 , activation = 'relu'))
model1.add(Dense(42 , activation = 'relu'))
model1.add(Dense(14 , activation = 'relu'))
model1.add(Dropout(0.4))

model1.add(Dense(2 , activation = 'softmax'))

model1.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [11]:
import xgboost as xgb

params = {"objective": "binary:logistic","eval_metric": "auc" ,"booster": "gbtree",
          "eta": 0.1, "colsample_bytree": 0.6, "min_child_weight": 0.7,
          "tree_method": "exact", "seed": "1992"}

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
bst = xgb.train( param, dtrain, 1, watchlist )
ptrain = bst.predict(dtrain, output_margin=True)
ptest  = bst.predict(dtest, output_margin=True)
ptrain = ptrain.reshape(-1, 1)
ptest = ptest.reshape(-1, 1)
dtrain.set_base_margin(ptrain)
dtest.set_base_margin(ptest)
print ('this is result of running from initial prediction')
bst = xgb.train( param, dtrain, 100, watchlist )

In [31]:
ptrain = predictors[:train_size].reshape(-1,1)

In [26]:
#ptrain.shape
target[7570777]

0

In [15]:
train_size = len(data) - len(test) - 200000
train_size
valid_size = len(data) - len(test)
valid_size

7570688

In [16]:
len(data)-7570688


19918

In [17]:
train_size - valid_size

-200000

In [12]:
predictors = data.iloc[:6000000,38:856].as_matrix()

In [49]:
predictors1 = data.iloc[:valid_size,38:856].as_matrix()
validset = data.iloc[train_size:valid_size,38:856].as_matrix()
testset = data.iloc[valid_size:,38:856].as_matrix()
dtrain = xgb.DMatrix(predictors, target[0:6000000])
dvalid = xgb.DMatrix(validset, target[train_size+1:])
dtest = xgb.DMatrix(testset)
dtrain = xgb.DMatrix(data.iloc[:3000000,38:856], target[0:3000000])

In [50]:
len(predictors1)

1570688

In [21]:

#data.iloc[:,38:856]
#dtest = xgb.DMatrix(test[features])

In [51]:
dtrain1 = xgb.DMatrix(predictors1, target[6000000:valid_size])

In [14]:
nrounds = 300
watchlist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, num_boost_round=nrounds, evals=watchlist, verbose_eval=20)

[0]	train-auc:0.721704
[20]	train-auc:0.742705
[40]	train-auc:0.749538
[60]	train-auc:0.753593
[80]	train-auc:0.756754
[100]	train-auc:0.758916
[120]	train-auc:0.760635
[140]	train-auc:0.762106
[160]	train-auc:0.763278
[180]	train-auc:0.76427
[200]	train-auc:0.765442
[220]	train-auc:0.766135
[240]	train-auc:0.767017
[260]	train-auc:0.767704
[280]	train-auc:0.768637


In [22]:
pred=bst.predict(dvalid)
from sklearn.metrics import roc_auc_score
roc_auc_score(target[train_size+1:], pred)

0.76924720080951736

In [ ]:
fpr, tpr, _ = roc_curve(target[train_size+1:], pred)
roc_auc = auc(fpr, tpr)

res.append(roc_auc)

print np.mean(roc_auc)

In [23]:
test_preds = bst.predict(dtest)
submit = pd.DataFrame({'sample_id': test['sample_id'], 'is_listened': test_preds})
submit.to_csv("DSG_xgb_50_300.csv", index=False)

In [52]:
train_preds = bst.predict(dtrain)
train_preds1 = bst.predict(dtrain1)

In [53]:
len(train_preds1)

1570688

In [57]:
#submit = pd.DataFrame({ 'is_listened': train_preds})
#submit1 = pd.DataFrame({ is_listened': train_preds1})
sub = pd.DataFrame(train_preds, columns=['xgb_50_300'])
sub1 = pd.DataFrame(train_preds1, columns=['xgb_50_300'])

In [58]:
sub=pd.concat([sub,sub1],axis=0)

In [59]:
len(sub)

7570688

In [60]:
sub.to_csv("xgb_50_300_train.csv", index=False)

In [26]:
train_preds

array([ 0.48920366,  0.56624347,  0.43722993, ...,  0.82246929,
        0.69349277,  0.82484359], dtype=float32)

In [73]:
# load json and create model
from keras.models import model_from_json
json_file = open('Keras_21054.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Keras_21054.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
b_tr = loaded_model.predict_proba(predictors, verbose=0)[:,1]
b_tr1 = loaded_model.predict_proba(predictors1, verbose=0)[:,1]